In [2]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Paso 1: Recopilación y preparación de los datos


In [3]:
dataset = pd.read_csv("/content/drive/MyDrive/Colab/Colab Archive/2020-07-20/BERT_sentiment_IMDB_Dataset.csv")
documents = dataset["review"]
labels = dataset["sentiment"]

# Paso 2: Construcción del diccionario de palabras emocionales utilizando WordNet


In [4]:
emotional_words = set()
for synset in wordnet.all_synsets():
    for lemma in synset.lemmas():
        if lemma.antonyms():
            emotional_words.add(lemma.name())

# Paso 3: Construcción de una matriz de documento de palabras utilizando TF-IDF


In [5]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)
#print("Matriz vectorial:")
#print(X.toarray())

# Paso 4: Función de peso (no se implementa en este código)


# Paso 5 y 6: SVD (Descomposición de valores singulares) y Reducción utilizando LSA


In [6]:
lsa = TruncatedSVD(n_components=100)  # Aumenta el número de componentes principales
X_lsa = lsa.fit_transform(X)

# Paso 7: Cálculo de la puntuación emocional
## División de los datos en conjuntos de entrenamiento y prueba


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_lsa, labels, test_size=0.2, random_state=42)


## Conversión de etiquetas de clase a valores numéricos

In [8]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


## Ajuste de hiperparámetros del Random Forest

In [9]:
random_forest = RandomForestClassifier(max_depth=10, min_samples_split=5)
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_split=5)

## Predicción en el conjunto de entrenamiento y cálculo de la pérdida y precisión

In [10]:
y_train_pred = random_forest.predict(X_train)
train_loss = log_loss(y_train, y_train_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)

## Predicción en el conjunto de prueba y cálculo de la pérdida y precisión


In [11]:
y_test_pred = random_forest.predict(X_test)
test_loss = log_loss(y_test, y_test_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)


# Paso 8: Interpretación y análisis de los resultados

In [12]:
print("\nPérdida en el conjunto de entrenamiento:", train_loss)
print("Precisión en el conjunto de entrenamiento:", train_accuracy)
print("\nPérdida en el conjunto de prueba:", test_loss)
print("Precisión en el conjunto de prueba:", test_accuracy)


Pérdida en el conjunto de entrenamiento: 4.080141563648061
Precisión en el conjunto de entrenamiento: 0.8868

Pérdida en el conjunto de prueba: 7.594397769086984
Precisión en el conjunto de prueba: 0.7893


# Probar con una oracion

In [13]:
def classifySentiment(text):
    # Paso 1: Preprocesamiento del texto de entrada
    processed_text = vectorizer.transform([text])

    # Paso 2: Reducción de dimensionalidad utilizando SVD (LSA)
    text_lsa = lsa.transform(processed_text)

    # Paso 3: Clasificación de polaridad utilizando el modelo de Random Forest
    sentiment_label = random_forest.predict(text_lsa)
    sentiment = label_encoder.inverse_transform(sentiment_label)[0]

    # Paso 4: Devolución de la clasificación de polaridad
    return sentiment

## resultado

In [14]:
review_text = "Avengers: Infinity War at least had the good taste to abstain from Jeremy Renner. No such luck in Endgame."
sentiment = classifySentiment(review_text)
print(review_text)
print("Sentiment:", sentiment)

Avengers: Infinity War at least had the good taste to abstain from Jeremy Renner. No such luck in Endgame.
Sentiment: negative
